<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Wurst Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Karin Treyer</a> (PSI)
                    <br><br>
                    Duration: 1 hour 30 minutes.
                </center>
            </td>
        </tr>
    </div>

Github repository: [https://github.com/polca/wurst](https://github.com/polca/wurst)

Documentation: [https://wurst.readthedocs.io/](https://wurst.readthedocs.io/)

Wurst authors: Chris Mutel, Brian Cox

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>. From the user end side, very little differ between the two. The code executed throughout this notebook works with both versions... probably.
</div>


### Scenario: xxxx

#### Background:



#### Objective:
To xxx

### 1. Import:

In [ ]:
import wurst
import wurst.searching as ws
import bw2io, bw2data
import matplotlib.pyplot as plt

In [ ]:
bw2data.projects

In [ ]:
bw2data.projects.set_current("bw25")